# Apache Spark

Sistema diseñado para procesar datos de manera distribuida sobre clusters. Spark puede procesar cantidades de datos en el orden de terabytes incluso petabytes.

El concepto de Spark es imaginar un cluster como una memoria gigante, la memoria resultante de de combinar las memorias de todos los clusters. Se prioriza el uso de memoria y consigue ser muy rápido el procesamiento de la inforamción, mayor que si utilizara MapReduce (Google).

Spark utiliza MapReduce para algunas ateas de clasificación mediante regresión. Apache Spark esta implementado en Scala que es ejecutado en la máquina virtual de Java. Además de Spark ofrece interfaces de programación para Java, Python y R.

### Caracteristicas

* Velocidad de procesamiento
* Soporte multilenguaje
* Análisis avanzado

Se puede desarrollar en tres maneras:
1. solo : standalone. Utiliza como base HDFS y encima se encuentra spark.
2. Hadoop (Yarn, adminsitrador de recursos) : Base HDFS -> Yarn/Mesos(kernel administrador del cluster) -> Spark
3. Spark con MapReduce (Spark In MapReduce, SIMR) : Base HDFS -> MapReduce y dentro se encuentra Spark

![apache-spark.jpg](apache-spark.jpg)

## Componentes de Spark

![apache-spark-core.jpg](apache-spark-core.jpg)

Cuando se ejecuta Spark en un ambiente distribuida se distinguen dos tipos de procesos: driver y executor. Un proceso driver conectado a 3 procesos excutor localizados en dos nodos del cluster: